In [1]:
import numpy as np
import pandas as pd

##### (Skeleton using chunks)

### Load the data

In [2]:
DATA_PATH = './Data/'
FILE2016 = DATA_PATH + 'quotes-2020.json.bz2'
df_base = pd.read_json(FILE2016, lines=True, compression='bz2', nrows=10000)

### Create a copy to work with

In [3]:
df = df_base.copy()
df

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2020-01-28-000082,[ D ] espite the efforts of the partners to cr...,None,[],2020-01-28 08:04:05,1,"[[None, 0.7272], [Prime Minister Netanyahu, 0....",[http://israelnationalnews.com/News/News.aspx/...,E
1,2020-01-16-000088,[ Department of Homeland Security ] was livid ...,Sue Myrick,[Q367796],2020-01-16 12:00:13,1,"[[Sue Myrick, 0.8867], [None, 0.0992], [Ron Wy...",[http://thehill.com/opinion/international/4782...,E
2,2020-02-10-000142,... He (Madhav) also disclosed that the illega...,None,[],2020-02-10 23:45:54,1,"[[None, 0.8926], [Prakash Rai, 0.1074]]",[https://indianexpress.com/article/business/ec...,E
3,2020-02-15-000053,"... [ I ] f it gets to the floor,",None,[],2020-02-15 14:12:51,2,"[[None, 0.581], [Andy Harris, 0.4191]]",[https://patriotpost.us/opinion/68622-trump-bu...,E
4,2020-01-24-000168,[ I met them ] when they just turned 4 and 7. ...,Meghan King Edmonds,[Q20684375],2020-01-24 20:37:09,4,"[[Meghan King Edmonds, 0.5446], [None, 0.2705]...",[https://people.com/parents/meghan-king-edmond...,E
...,...,...,...,...,...,...,...,...,...
9995,2020-03-02-078576,"Why now, 2021? Here's why,",Michael J. Graham,[Q6831418],2020-03-02 21:03:20,1,"[[Michael J. Graham, 0.6261], [None, 0.3739]]",[https://thecourier.com/ohio-news/2020/03/02/p...,E
9996,2020-04-14-074488,Why should a running back be treated less than...,Mark Ingram,[Q452154],2020-04-14 22:03:37,1,"[[Mark Ingram, 0.5275], [None, 0.4657], [Chris...",[http://registercitizen.com/sports/article/Ing...,E
9997,2020-04-15-078413,"Why would I leave here where I'm healthy, to t...",Pauline Pearce,[Q55076551],2020-04-15 10:10:33,1,"[[Pauline Pearce, 0.8222], [None, 0.1778]]",[https://www.bbc.co.uk/news/uk-england-london-...,E
9998,2020-01-18-055386,"Will he be back? And if so, will it be with th...",None,[],2020-01-18 05:30:00,1,"[[None, 0.8911], [Tom Brady, 0.0929], [Logan R...",[http://www.mywebtimes.com/2020/01/14/full-cou...,E


### Check uniqueness of the keys

In [4]:
print(df.quoteID.is_unique)
df = df.drop_duplicates(subset=['quoteID'], keep='first')

True


In [5]:
def process_chunk_uniqueid(chunk):
    print(f'Processing chunk with {len(chunk)} rows')
    print(chunk.quoteID.is_unique, '\n')
    if not(chunk.quoteID.is_unique):
        chunk = chunk.drop_duplicates(subset=['quoteID'], keep='first')

with pd.read_json(FILE2016, lines=True, compression='bz2', chunksize=1000) as df_reader:
    for chunk in df_reader:
        process_chunk_uniqueid(chunk)

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 1000 rows
True 

Processing chunk with 100

KeyboardInterrupt: 

In [6]:
process_chunk_uniqueid(df)

Processing chunk with 10000 rows
True 



### Remove `Nan` & empty quotes 

Todo:
- Use the isna() and isnull() to check to nan/empty list and remove any true values priting the number it may have removed --> Probs gonna be zero tbh 

In [7]:
df[df['quotation']=='None'].index

Int64Index([], dtype='int64')

In [27]:
# this is to verify the nan values 
print("We find {} nan values", df["quotation"].isna().sum())


We find {} nan values 0


In [30]:
# This is for empty list
df["quotation"].isnull()

0

### Remove `None` speakers

#### Automated using chunks

In [35]:
def process_chunk_removeNone(chunk):
    tot_length = 0
    print(f'Processing chunk with {len(chunk)} rows')
    chunk = chunk.drop(chunk[chunk['speaker']=='None'].index)
    chunk = chunk.reset_index(drop=True)
    tot_length = tot_length + len(chunk)
    print('Length of the chunk is now', len(chunk), 'rows\n')
    return tot_length, chunk
    

with pd.read_json(FILE2016, lines=True, compression='bz2', chunksize=10000) as df_reader:
    for chunk in df_reader:
        length_df = process_chunk_removeNone(chunk)

Processing chunk with 10000 rows
Length of the chunk is now 6575 rows

Processing chunk with 10000 rows
Length of the chunk is now 6554 rows

Processing chunk with 10000 rows
Length of the chunk is now 6546 rows



KeyboardInterrupt: 

#### Test over the downloaded dataframe

In [39]:
l, df = process_chunk_removeNone(df)
df
#don't understand why the output is not of 6575 rows :(

# It is not bevause the scope of df is only in the function thus you need to return it :)
# See how the function changed now ? 
# Now it should work, but I wold be very careful using the same name here
# If run twice the print text would change ofc

Processing chunk with 6575 rows
Length of the chunk is now 6575 rows



,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2020-01-16-000088,[ Department of Homeland Security ] was livid ...,Sue Myrick,[Q367796],2020-01-16 12:00:13,1,"[[Sue Myrick, 0.8867], [None, 0.0992], [Ron Wy...",[http://thehill.com/opinion/international/4782...,E
1,2020-01-24-000168,[ I met them ] when they just turned 4 and 7. ...,Meghan King Edmonds,[Q20684375],2020-01-24 20:37:09,4,"[[Meghan King Edmonds, 0.5446], [None, 0.2705]...",[https://people.com/parents/meghan-king-edmond...,E
2,2020-01-17-000357,[ The delay ] will have an impact [ on Slough ...,Dexter Smith,[Q5268447],2020-01-17 13:03:00,1,"[[Dexter Smith, 0.924], [None, 0.076]]",[http://www.sloughexpress.co.uk/gallery/slough...,E
3,2020-04-02-000239,[ The scheme ] treats addiction as an illness ...,Barry Coppinger,[Q4864119],2020-04-02 14:18:20,1,"[[Barry Coppinger, 0.9017], [None, 0.0983]]",[http://www.theweek.co.uk/106479/why-police-ar...,E
4,2020-03-19-000276,[ These ] actions will allow households who ha...,Ben Carson,[Q816459],2020-03-19 19:14:00,1,"[[Ben Carson, 0.9227], [None, 0.0773]]",[https://mortgageorb.com/hud-fha-suspend-forec...,E
...,...,...,...,...,...,...,...,...,...
6570,2020-02-17-085956,"whoever is the interested player,",Hardeep Singh Puri,[Q5655835],2020-02-17 10:15:26,6,"[[Hardeep Singh Puri, 0.9405], [None, 0.0595]]",[https://www.livemint.com/news/india/-air-indi...,E
6571,2020-03-02-078576,"Why now, 2021? Here's why,",Michael J. Graham,[Q6831418],2020-03-02 21:03:20,1,"[[Michael J. Graham, 0.6261], [None, 0.3739]]",[https://thecourier.com/ohio-news/2020/03/02/p...,E
6572,2020-04-14-074488,Why should a running back be treated less than...,Mark Ingram,[Q452154],2020-04-14 22:03:37,1,"[[Mark Ingram, 0.5275], [None, 0.4657], [Chris...",[http://registercitizen.com/sports/article/Ing...,E
6573,2020-04-15-078413,"Why would I leave here where I'm healthy, to t...",Pauline Pearce,[Q55076551],2020-04-15 10:10:33,1,"[[Pauline Pearce, 0.8222], [None, 0.1778]]",[https://www.bbc.co.uk/news/uk-england-london-...,E


In [37]:
ok

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2020-01-16-000088,[ Department of Homeland Security ] was livid ...,Sue Myrick,[Q367796],2020-01-16 12:00:13,1,"[[Sue Myrick, 0.8867], [None, 0.0992], [Ron Wy...",[http://thehill.com/opinion/international/4782...,E
1,2020-01-24-000168,[ I met them ] when they just turned 4 and 7. ...,Meghan King Edmonds,[Q20684375],2020-01-24 20:37:09,4,"[[Meghan King Edmonds, 0.5446], [None, 0.2705]...",[https://people.com/parents/meghan-king-edmond...,E
2,2020-01-17-000357,[ The delay ] will have an impact [ on Slough ...,Dexter Smith,[Q5268447],2020-01-17 13:03:00,1,"[[Dexter Smith, 0.924], [None, 0.076]]",[http://www.sloughexpress.co.uk/gallery/slough...,E
3,2020-04-02-000239,[ The scheme ] treats addiction as an illness ...,Barry Coppinger,[Q4864119],2020-04-02 14:18:20,1,"[[Barry Coppinger, 0.9017], [None, 0.0983]]",[http://www.theweek.co.uk/106479/why-police-ar...,E
4,2020-03-19-000276,[ These ] actions will allow households who ha...,Ben Carson,[Q816459],2020-03-19 19:14:00,1,"[[Ben Carson, 0.9227], [None, 0.0773]]",[https://mortgageorb.com/hud-fha-suspend-forec...,E
...,...,...,...,...,...,...,...,...,...
6570,2020-02-17-085956,"whoever is the interested player,",Hardeep Singh Puri,[Q5655835],2020-02-17 10:15:26,6,"[[Hardeep Singh Puri, 0.9405], [None, 0.0595]]",[https://www.livemint.com/news/india/-air-indi...,E
6571,2020-03-02-078576,"Why now, 2021? Here's why,",Michael J. Graham,[Q6831418],2020-03-02 21:03:20,1,"[[Michael J. Graham, 0.6261], [None, 0.3739]]",[https://thecourier.com/ohio-news/2020/03/02/p...,E
6572,2020-04-14-074488,Why should a running back be treated less than...,Mark Ingram,[Q452154],2020-04-14 22:03:37,1,"[[Mark Ingram, 0.5275], [None, 0.4657], [Chris...",[http://registercitizen.com/sports/article/Ing...,E
6573,2020-04-15-078413,"Why would I leave here where I'm healthy, to t...",Pauline Pearce,[Q55076551],2020-04-15 10:10:33,1,"[[Pauline Pearce, 0.8222], [None, 0.1778]]",[https://www.bbc.co.uk/news/uk-england-london-...,E


#### Applying to a portion of the dataframe (to compare with the cell above)
gg

In [40]:
df_speaker = df.drop(df[df['speaker']=='None'].index)
df_speaker = df_speaker.reset_index(drop=True)
df_speaker

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2020-01-16-000088,[ Department of Homeland Security ] was livid ...,Sue Myrick,[Q367796],2020-01-16 12:00:13,1,"[[Sue Myrick, 0.8867], [None, 0.0992], [Ron Wy...",[http://thehill.com/opinion/international/4782...,E
1,2020-01-24-000168,[ I met them ] when they just turned 4 and 7. ...,Meghan King Edmonds,[Q20684375],2020-01-24 20:37:09,4,"[[Meghan King Edmonds, 0.5446], [None, 0.2705]...",[https://people.com/parents/meghan-king-edmond...,E
2,2020-01-17-000357,[ The delay ] will have an impact [ on Slough ...,Dexter Smith,[Q5268447],2020-01-17 13:03:00,1,"[[Dexter Smith, 0.924], [None, 0.076]]",[http://www.sloughexpress.co.uk/gallery/slough...,E
3,2020-04-02-000239,[ The scheme ] treats addiction as an illness ...,Barry Coppinger,[Q4864119],2020-04-02 14:18:20,1,"[[Barry Coppinger, 0.9017], [None, 0.0983]]",[http://www.theweek.co.uk/106479/why-police-ar...,E
4,2020-03-19-000276,[ These ] actions will allow households who ha...,Ben Carson,[Q816459],2020-03-19 19:14:00,1,"[[Ben Carson, 0.9227], [None, 0.0773]]",[https://mortgageorb.com/hud-fha-suspend-forec...,E
...,...,...,...,...,...,...,...,...,...
6570,2020-02-17-085956,"whoever is the interested player,",Hardeep Singh Puri,[Q5655835],2020-02-17 10:15:26,6,"[[Hardeep Singh Puri, 0.9405], [None, 0.0595]]",[https://www.livemint.com/news/india/-air-indi...,E
6571,2020-03-02-078576,"Why now, 2021? Here's why,",Michael J. Graham,[Q6831418],2020-03-02 21:03:20,1,"[[Michael J. Graham, 0.6261], [None, 0.3739]]",[https://thecourier.com/ohio-news/2020/03/02/p...,E
6572,2020-04-14-074488,Why should a running back be treated less than...,Mark Ingram,[Q452154],2020-04-14 22:03:37,1,"[[Mark Ingram, 0.5275], [None, 0.4657], [Chris...",[http://registercitizen.com/sports/article/Ing...,E
6573,2020-04-15-078413,"Why would I leave here where I'm healthy, to t...",Pauline Pearce,[Q55076551],2020-04-15 10:10:33,1,"[[Pauline Pearce, 0.8222], [None, 0.1778]]",[https://www.bbc.co.uk/news/uk-england-london-...,E


In [41]:
print('In total,', len(df)-len(df_speaker), 'rows had no speaker and were thus removed')

In total, 0 rows had no speaker and were thus removed


### Remove speakers for which the probability is smaller than a threshold

#### Automated fashion

In [43]:
def process_chunk_probas(chunk):
    print(f'Processing chunk with {len(chunk)} rows')
    #Define threshold (50%)
    threshold_probas = 0.5
    
    #Gather the first probability for each row
    string_probas, nber_probas = [], []
    string_probas = [chunk['probas'][i][0][1] for i in chunk.index]
    nber_probas = list(map(float, string_probas))
    series_probas = pd.Series(nber_probas, dtype='float64', index=chunk.index)
    
    #Check if the probability is larger than the threshold, if not remove corresponding index
    chunk = chunk.drop(series_probas[series_probas < threshold_probas].index)
    chunk = chunk.reset_index(drop=True)
    
    print('Length of the chunk is now', len(chunk), 'rows\n')
    return chunk

with pd.read_json(FILE2016, lines=True, compression='bz2', chunksize=10000) as df_reader:
    for chunk in df_reader:
        dropped_prob_chunk = process_chunk_probas(chunk)

Processing chunk with 10000 rows
Length of the chunk is now 9525 rows

Processing chunk with 10000 rows
Length of the chunk is now 9540 rows

Processing chunk with 10000 rows
Length of the chunk is now 9500 rows

Processing chunk with 10000 rows
Length of the chunk is now 9506 rows

Processing chunk with 10000 rows
Length of the chunk is now 9528 rows



KeyboardInterrupt: 

##### Test over the downloaded datafame

In [44]:
df = process_chunk_probas(df)
df
#once again, seems not to work...
# This is because you were not returning anything; so now the process_chunj_probas 
# actually returns the updated df

Processing chunk with 6575 rows
Length of the chunk is now 6282 rows



,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2020-01-16-000088,[ Department of Homeland Security ] was livid ...,Sue Myrick,[Q367796],2020-01-16 12:00:13,1,"[[Sue Myrick, 0.8867], [None, 0.0992], [Ron Wy...",[http://thehill.com/opinion/international/4782...,E
1,2020-01-24-000168,[ I met them ] when they just turned 4 and 7. ...,Meghan King Edmonds,[Q20684375],2020-01-24 20:37:09,4,"[[Meghan King Edmonds, 0.5446], [None, 0.2705]...",[https://people.com/parents/meghan-king-edmond...,E
2,2020-01-17-000357,[ The delay ] will have an impact [ on Slough ...,Dexter Smith,[Q5268447],2020-01-17 13:03:00,1,"[[Dexter Smith, 0.924], [None, 0.076]]",[http://www.sloughexpress.co.uk/gallery/slough...,E
3,2020-04-02-000239,[ The scheme ] treats addiction as an illness ...,Barry Coppinger,[Q4864119],2020-04-02 14:18:20,1,"[[Barry Coppinger, 0.9017], [None, 0.0983]]",[http://www.theweek.co.uk/106479/why-police-ar...,E
4,2020-03-19-000276,[ These ] actions will allow households who ha...,Ben Carson,[Q816459],2020-03-19 19:14:00,1,"[[Ben Carson, 0.9227], [None, 0.0773]]",[https://mortgageorb.com/hud-fha-suspend-forec...,E
...,...,...,...,...,...,...,...,...,...
6277,2020-02-17-085956,"whoever is the interested player,",Hardeep Singh Puri,[Q5655835],2020-02-17 10:15:26,6,"[[Hardeep Singh Puri, 0.9405], [None, 0.0595]]",[https://www.livemint.com/news/india/-air-indi...,E
6278,2020-03-02-078576,"Why now, 2021? Here's why,",Michael J. Graham,[Q6831418],2020-03-02 21:03:20,1,"[[Michael J. Graham, 0.6261], [None, 0.3739]]",[https://thecourier.com/ohio-news/2020/03/02/p...,E
6279,2020-04-14-074488,Why should a running back be treated less than...,Mark Ingram,[Q452154],2020-04-14 22:03:37,1,"[[Mark Ingram, 0.5275], [None, 0.4657], [Chris...",[http://registercitizen.com/sports/article/Ing...,E
6280,2020-04-15-078413,"Why would I leave here where I'm healthy, to t...",Pauline Pearce,[Q55076551],2020-04-15 10:10:33,1,"[[Pauline Pearce, 0.8222], [None, 0.1778]]",[https://www.bbc.co.uk/news/uk-england-london-...,E


#### Applying to a portion of the dataframe

In [45]:
#Define threshold (50%)
threshold_probas = 0.5

#Gather the first probability for each row
string_probas = [df['probas'][i][0][1] for i in range(len(df))]
nber_probas = list(map(float, string_probas))
series_probas = pd.Series(nber_probas)

#Check if the probability is larger than the threshold, if not remove the row
df_probas = df.drop(series_probas[series_probas < threshold_probas].index)
df_probas = df_probas.reset_index(drop=True)
df_probas

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2020-01-16-000088,[ Department of Homeland Security ] was livid ...,Sue Myrick,[Q367796],2020-01-16 12:00:13,1,"[[Sue Myrick, 0.8867], [None, 0.0992], [Ron Wy...",[http://thehill.com/opinion/international/4782...,E
1,2020-01-24-000168,[ I met them ] when they just turned 4 and 7. ...,Meghan King Edmonds,[Q20684375],2020-01-24 20:37:09,4,"[[Meghan King Edmonds, 0.5446], [None, 0.2705]...",[https://people.com/parents/meghan-king-edmond...,E
2,2020-01-17-000357,[ The delay ] will have an impact [ on Slough ...,Dexter Smith,[Q5268447],2020-01-17 13:03:00,1,"[[Dexter Smith, 0.924], [None, 0.076]]",[http://www.sloughexpress.co.uk/gallery/slough...,E
3,2020-04-02-000239,[ The scheme ] treats addiction as an illness ...,Barry Coppinger,[Q4864119],2020-04-02 14:18:20,1,"[[Barry Coppinger, 0.9017], [None, 0.0983]]",[http://www.theweek.co.uk/106479/why-police-ar...,E
4,2020-03-19-000276,[ These ] actions will allow households who ha...,Ben Carson,[Q816459],2020-03-19 19:14:00,1,"[[Ben Carson, 0.9227], [None, 0.0773]]",[https://mortgageorb.com/hud-fha-suspend-forec...,E
...,...,...,...,...,...,...,...,...,...
6277,2020-02-17-085956,"whoever is the interested player,",Hardeep Singh Puri,[Q5655835],2020-02-17 10:15:26,6,"[[Hardeep Singh Puri, 0.9405], [None, 0.0595]]",[https://www.livemint.com/news/india/-air-indi...,E
6278,2020-03-02-078576,"Why now, 2021? Here's why,",Michael J. Graham,[Q6831418],2020-03-02 21:03:20,1,"[[Michael J. Graham, 0.6261], [None, 0.3739]]",[https://thecourier.com/ohio-news/2020/03/02/p...,E
6279,2020-04-14-074488,Why should a running back be treated less than...,Mark Ingram,[Q452154],2020-04-14 22:03:37,1,"[[Mark Ingram, 0.5275], [None, 0.4657], [Chris...",[http://registercitizen.com/sports/article/Ing...,E
6280,2020-04-15-078413,"Why would I leave here where I'm healthy, to t...",Pauline Pearce,[Q55076551],2020-04-15 10:10:33,1,"[[Pauline Pearce, 0.8222], [None, 0.1778]]",[https://www.bbc.co.uk/news/uk-england-london-...,E


In [46]:
print('In total,', len(df)-len(df_probas), 'speakers had a probality smaller than', threshold_probas, 'and were thus removed')

In total, 0 speakers had a probality smaller than 0.5 and were thus removed


#### Think to other scenarios

In [ ]:
#If all speakers are women but with proba < threshold, keep the row ?
#might cause some problem for interpretation (country of origin of the speaker)


#### Additional notes Lavi 1.11.

- I don't think we should keep the speakers if proba < threshold, because like you state above, this will create additional problems

Possible additional cleaning may include:
- checking nan values 
- verify absurd QuoteID or QID 
- make sure quotes are non empty

#### Additional notes Sélène 1.11.
I saw in 2019 a quote that looks wrong ("% 9: D: D @? 6 H2J H6 E9@F89E H6 4@F=5 8: G6 3..."). Idk if possible but we could perhaps remove quotes with only weird symbols as part of cleaning?

--> This was a None speaker but indeed would be good, some function for sure exist already made :)